In [1]:
import pandas as pd
import numpy as np
import sys
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 550)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 300)
pd.options.mode.chained_assignment = None  # default='warn'

## Vascular Access

In [51]:
def compute_lesion_vessels(df, column='L1'):
    lesion = column + ' Lesion Vessel'
    segment = column + ' Segment Number'
    if (column == 'L10'):
        stent = column + ' Total Number of Stents Placed'
    else:
        stent = column + ' Total Number of Stents Placed'
    df[lesion] = np.where(df[segment]==5, 'Left Main', np.where(df[segment].isin([6,7,8,9,10]),'LAD',np.where(df[segment].isin([11,12,13,14,15]),'LCx',np.where(df[segment].isin([1,2,3,4,5]),'RCA','NA'))))
    df[stent] = pd.to_numeric(df[stent], errors='coerce').fillna(0)
    return df
def init(file):
    df = pd.read_excel(file)
    df['Procedure_Date'] = df['Procedure Date']
    df2_columns = df.filter(like='Number of Stents Placed').columns.values
    df2 = df[df2_columns]
    df['stent_placed'] = ['Yes' if x >= 1 else 'No' for x in np.sum(((df2.values == 1)|(df2.values == 2)), 1)]
    df['stent_number_lesions'] = [x if x >= 1 else 'No' for x in np.sum(((df2.values == 1)|(df2.values == 2)), 1)]
    df2_columns = df.filter(like='Intervention Performed').columns.values
    df2 = df[df2_columns]
    display(df2)
    df['ffr_or_ivus_or_oct_or_pci'] = ['Yes' if x >= 1 else 'No' for x in np.sum(df2.values == 'Yes', 1)]
    df2_columns = df.filter(like='Balloon Angioplasty').columns.values
    df2 = df[df2_columns]
    df['balloon_angioplasty'] = ['Yes' if x >= 1 else 'No' for x in np.sum((df2.values == 'Yes'), 1)]
    #df3 = df2.loc[df2.apply(lambda x: 'Yes' in x.values, axis=1).any()]
    df['intervention_performed'] = np.where((df['stent_placed']=='Yes')|(df['balloon_angioplasty']=='Yes'), 1, 0)


    df2_columns = df.filter(like='FFR').columns.values
    df2 = df[df2_columns]
    df['ffr_performed'] = ['Yes' if x >= 1 else 'No' for x in np.sum((df2.values == 'Yes'), 1)]

    df2_columns = df.filter(like='IVUS').columns.values
    df2 = df[df2_columns]
    df['ivus_performed'] = ['Yes' if x >= 1 else 'No' for x in np.sum((df2.values == 'Yes'), 1)]

    df2_columns = df.filter(like='OCT').columns.values
    df2 = df[df2_columns]
    df['oct_performed'] = ['Yes' if x >= 1 else 'No' for x in np.sum((df2.values == 'Yes'), 1)]


    df2_columns = df.filter(like='Guideliner Used').columns.values
    df2 = df[df2_columns]
    df['guideliner_used'] = ['Yes' if x >= 1 else 'No' for x in np.sum((df2.values == 'Yes'), 1)]



    df['hemodynamic_instability'] = np.where(((df['Intubated pre-cath']=='yes')|(df['ROSC']!='no')|(df['Procedural Complications Arrest']=='Yes')|(df['Procedural Complications Shock']=='Yes')), 'Yes','No')
    #Defined as ROSC, or intubated or periprocedural arrest or shock


    df2_columns = df.filter(like='Thrombectomy').columns.values
    df2 = df[df2_columns]
    df['thrombectomy_used'] = ['Yes' if x >= 1 else 'No' for x in np.sum((df2.values == 'Yes'), 1)]
    df2_columns = df.filter(like='Distal protection').columns.values
    df2 = df[df2_columns]
    df['distal_protection_used'] = ['Yes' if x >= 1 else 'No' for x in np.sum((df2.values == 'Yes'), 1)]
    df2_columns = df.filter(like='Rotablator').columns.values
    df2 = df[df2_columns]
    df['rotablator_used'] = ['Yes' if x >= 1 else 'No' for x in np.sum((df2.values == 'Yes'), 1)]
    df2_columns = df.filter(like='Cutting balloon').columns.values
    df2 = df[df2_columns]
    df['cutting_balloon_used'] = ['Yes' if x >= 1 else 'No' for x in np.sum((df2.values == 'Yes'), 1)]
    for lesions in ['L1', 'L2', 'L3','L4','L5','L6','L7','L8','L9','L10']:
        df = compute_lesion_vessels(df, lesions)

    display(df.stent_placed.value_counts())
    display(df.ffr_performed.value_counts())
    display(df.ivus_performed.value_counts())
    display(df.oct_performed.value_counts())

    display(df.balloon_angioplasty.value_counts())
    display(df.intervention_performed.value_counts())
    display(df.STEMI.value_counts())
    #display(df.STEMI_Type.value_counts())
    display(df.ffr_or_ivus_or_oct_or_pci.value_counts())

    display(df.hemodynamic_instability.value_counts())
    #display(df.Procedural_Complications_Present.value_counts())
    return df

In [52]:
#print("Values between the two dates", df.Procedure_Date.min(),  df.Procedure_Date.max())
#print("Vascular Access", df.Access_Site_Utilized.value_counts())

## Diagnostics vs PCI

In [53]:
def compute_multi_vessel(df_intervention):
    for index, row in df_intervention.iterrows():
        column_list =  ['L1','L2', 'L3','L4','L5','L6','L7','L8','L9','L10']
        for column in column_list:
            lesion = column + ' Lesion Vessel'
            if (column == 'L10'):
                stent = column + ' Total Number of Stents Placed'
            else:
                stent = column + ' Total Number of Stents Placed'
            balloon = column + ' Balloon Angioplasty?'
            if (row[stent]==1) | (row[stent]==2)|(row[balloon]=='Yes'):
                #found a vessel that was stented
                list_copy = column_list
                list_copy.remove(column)
                vessel_stented = row[lesion]

                for filtered_column in list_copy:
                    filtered_lesion = filtered_column + ' Lesion Vessel'
                    if (filtered_column == 'L10'):
                        filtered_stent = filtered_column + ' Total Number of Stents Placed'
                    else:
                        filtered_stent = filtered_column + ' Total Number of Stents Placed'

                    filtered_balloon = filtered_column + ' Balloon Angioplasty?'
                    if (row[filtered_stent]==1) | (row[filtered_stent]==2)|(row[filtered_balloon]=='Yes'):
                        if (vessel_stented) != row[filtered_lesion]:
                            df_intervention.at[index,'multi_vessel'] = True
                            break
    return df_intervention

In [54]:
def compute_bifurcation(df_intervention):
    #TODO : CLASSIFY MEDINA FOR BIFURCATIONS
    #TODO : CLASSIFY TECHNIQUE

    for index, row in df_intervention.iterrows():
        column_list =  ['L1','L2', 'L3','L4','L5','L6','L7','L8','L9','L10']
        for column in column_list:
            lesion = column + ' Lesion Vessel'
            if (column == 'L10'):
                stent = column + ' Total Number of Stents Placed'
            else:
                stent = column + ' Total Number of Stents Placed'
            balloon = column + ' Balloon Angioplasty?'
            bifurcation = column + ' Bifurcation Lesion'
           
            if (row[stent]==1) | (row[stent]==2)|(row[balloon]=='Yes'):
                if (row[bifurcation]=='Yes'):
                    df_intervention.at[index,'bifurcation_interv'] = True
                    break
                elif column=='L10':
                    df_intervention.at[index,'bifurcation_interv'] = False
    return df_intervention

def compute_left_main(df_intervention):
    #TODO : CLASSIFY MEDINA FOR BIFURCATIONS
    #TODO : CLASSIFY TECHNIQUE

    for index, row in df_intervention.iterrows():
        column_list =  ['L1','L2', 'L3','L4','L5','L6','L7','L8','L9','L10']
        for column in column_list:
            lesion = column + ' Lesion Vessel'
            if (column == 'L10'):
                stent = column + ' Total Number of Stents Placed'
            else:
                stent = column + ' Total Number of Stents Placed'
            balloon = column + ' Balloon Angioplasty?'
            bifurcation = column + ' Bifurcation Lesion'
           
            if (row[stent]==1) | (row[stent]==2)|(row[balloon]=='Yes'):
                if (row[lesion]=='Left Main'):
                    df_intervention.at[index,'lm_interv'] = True
                    break
                elif column=='L10':
                    df_intervention.at[index,'lm_interv'] = False
    return df_intervention

In [55]:
def generate_intervention_dataframe(df):
    df_intervention = df.loc[df['intervention_performed']==1].reset_index()

    df_intervention = compute_multi_vessel(df_intervention)
    df_intervention = compute_bifurcation(df_intervention)
    df_intervention = compute_left_main(df_intervention)
    df_intervention['complex_cad_case'] = np.where(((df_intervention['guideliner_used']=='Yes') | (df_intervention['thrombectomy_used']=='Yes') | (df_intervention['distal_protection_used']=='Yes') | (df_intervention['rotablator_used']=='Yes') | (df_intervention['cutting_balloon_used']=='Yes')| (df_intervention['lm_interv']==True) | (df_intervention['bifurcation_interv']==True)), 1, 0)
    return df_intervention

In [56]:
def generate_summary_dataframe(df, df_intervention):
    objective = []
    count = []

    objective.append('Radial Access')
    count.append(len(df.loc[df['Access Site Utilized']=='Radial']))
    objective.append('Femoral Access')
    count.append(len(df.loc[df['Access Site Utilized']=='Femoral']))
    objective.append('Brachial Access')
    count.append(len(df.loc[df['Access Site Utilized']=='Brachial']))
    objective.append('Number of diagnostic cases')
    count.append(len(df.loc[df['ffr_or_ivus_or_oct_or_pci']=='No']))
    objective.append('Number of FFR, IVUS, OCT or PCI cases')
    count.append(len(df.loc[df['ffr_or_ivus_or_oct_or_pci']=='Yes']))
    objective.append('Number of FFR, IVUS, OCT or PCI cases')
    count.append(len(df.loc[df['ffr_or_ivus_or_oct_or_pci']=='Yes']))
    objective.append('FFR cases')
    count.append(len(df.loc[df['ffr_performed']=='Yes']))
    objective.append('IVUS cases')
    count.append(len(df.loc[df['ivus_performed']=='Yes']))
    objective.append('OCT cases')
    count.append(len(df.loc[df['oct_performed']=='Yes']))
    objective.append('PCI cases')
    count.append(len(df.loc[df['stent_placed']=='Yes']))
    objective.append('STEMI')
    count.append(len(df.loc[df['STEMI']=='yes']))
    objective.append('Hemodynamic Instability [Shock, Arrest, ROSC, Intubated]')
    count.append(len(df.loc[df['hemodynamic_instability']=='Yes']))
    objective.append('Multivessel PCI')
    count.append(len(df_intervention.loc[df_intervention['multi_vessel']==True]))
    objective.append('Bifurcation PCI')
    count.append(len(df_intervention.loc[df_intervention['bifurcation_interv']==True]))
    objective.append('Left Main PCI')
    count.append(len(df_intervention.loc[df_intervention['lm_interv']==True]))
    objective.append('Complex CAD [Bifurcation, Calcifried, LM, CTO or thrombotic]')
    count.append(len(df_intervention.loc[df_intervention['complex_cad_case']==True]))
    df_count = pd.DataFrame(list(zip(objective, count)), 
               columns =['Objective', 'Number'])
    return df_count

In [57]:
def save_files (df, df_intervention, df_count):
    from datetime import date

    today = date.today()
    output_count = 'condensed_summary_' + str(today) +'.csv'
    intervention_output = 'interventions_' + str(today) +'.csv'
    overall_output = 'overall_' + str(today) +'.csv'
    df.to_csv(overall_output)
    df_intervention.to_csv(intervention_output)
    df_count.to_csv(output_count)

In [58]:
full_work('data/avram_june_03_01.xlsx')

,L1 Intervention Performed?,L2 Intervention Performed?,L3 Intervention Performed?,L4 Intervention Performed?,L5 Intervention Performed?,L6 Intervention Performed?,L7 Intervention Performed?,L8 Intervention Performed?,L9 Intervention Performed?,L10 Intervention Performed?
0,No,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,No,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
734,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
735,No,No,No,No,No,No,No,NaN,NaN,NaN
736,Yes,Yes,Yes,No,No,NaN,NaN,NaN,NaN,NaN
737,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


No     455
Yes    284
Name: stent_placed, dtype: int64

No     682
Yes     57
Name: ffr_performed, dtype: int64

No     714
Yes     25
Name: ivus_performed, dtype: int64

No     719
Yes     20
Name: oct_performed, dtype: int64

...

0    446
1    293
Name: intervention_performed, dtype: int64

no     288
yes     96
Name: STEMI, dtype: int64

No     386
Yes    353
Name: ffr_or_ivus_or_oct_or_pci, dtype: int64

No     716
Yes     23
Name: hemodynamic_instability, dtype: int64

Files generated successfully


In [22]:
def full_work(path):

    try:
        file = open(path, 'r')
        df = init(path)
        df_intervention = generate_intervention_dataframe(df)
        df_count = generate_summary_dataframe(df, df_intervention)
        save_files (df, df_intervention, df_count)
        return print('Files generated successfully')
    except IOError:
        return print('There was an error opening the file', path)
    

In [23]:
if __name__ == '__main__':
    globals()[sys.argv[1]](sys.argv[2])
    full_work(sys.argv[2])

KeyError: '-f'

In [24]:
def generate_file_for_cath_ef(file='data/verreault_04012021.xlsx'):
    from numpy import nan
    df = pd.read_excel(file)
    #display(df.head(n=5))
    try:
        df_1 = df.loc[df['LVEF']!='unknown'].sample(250)
        df_2 = df.loc[df['LVEF']=='unknown'].sample(50)
    except:
        try:
            df_1 = df.loc[df['LVEF']!='unknown'].sample(200)
            df_2 = df.loc[df['LVEF']=='unknown'].sample(100)
        except:
            try:
                df_1 = df.loc[df['LVEF']!='unknown'].sample(150)
                df_2 = df.loc[df['LVEF']=='unknown'].sample(75)
            except:
                df_1 = df.loc[df['LVEF']!='unknown'].sample(35)
                df_2 = df.loc[df['LVEF']=='unknown'].sample(215)
    df = pd.concat([df_1, df_2])
    try:
        df_f = df[['MRN','Date_of_Birth','Serial','SID','Procedure_Date','LVEF','LVEF_Determination_Method','LVEF_Determination_Date','Ejection Fraction','Creatinine - Baseline','Creatinine - Pre-procedure','Renal replacement?','Height','Weight']]
    except:
        print("MRN and DOB not available")
        df_f = df[['Serial','SID','Procedure_Date','LVEF','LVEF_Determination_Method','LVEF_Determination_Date','Ejection Fraction','Creatinine - Baseline','Creatinine - Pre-procedure','Renal replacement?','Height','Weight']]


    df_f['mrn'] = nan
    df_f['gender'] = nan
    df_f['Procedure_Date'] = pd.to_datetime(df_f['Procedure_Date'])
    df_f['lvef_modality_start_date'] = pd.to_datetime(df_f['Procedure_Date']) - pd.DateOffset(months=3)
    df_f['lvef_modality_end_date'] = pd.to_datetime(df_f['Procedure_Date']) + pd.DateOffset(months=3)
    df_f['lvef_modality_end_date'] = pd.to_datetime(df_f['Procedure_Date']) + pd.DateOffset(months=3)
    df_f['lvef_modality_1_type'] = nan
    df_f['lvef_modality_1_date'] = nan
    df_f['lvef_modality_1_value'] = nan
    df_f['lvef_modality_1_performed_at_uohi'] = nan
    df_f['lvef_modality_2_type'] = nan
    df_f['lvef_modality_2_date'] = nan
    df_f['lvef_modality_2_value'] = nan
    df_f['lvef_modality_2_performed_at_uohi'] = nan
    df_f['lvef_modality_3_type'] = nan
    df_f['lvef_modality_3_date'] = nan
    df_f['lvef_modality_3_value'] = nan
    df_f['lvef_modality_3_performed_at_uohi'] = nan
    df_f['creatine_baseline'] = nan
    df_f['creatine_baseline_date'] = nan
    df_f['creatine_post_procedure'] = nan
    df_f['creatine_post_procedure_date'] = nan
    
    try:
        df_f['mrn'] = df_f['MRN']
        df_f = df_f.drop(columns=['MRN'])
        df_f['Date_of_Birth'] = pd.to_datetime(df_f['Date_of_Birth'])
    except:
        df_f['Date_of_Birth'] = nan
    #display(df_f.head(n=5))
    df_f.to_csv(file.rstrip('.xlsx') + '_lvef.csv')
    return df_f

In [34]:
#df_f = generate_file_for_cath_ef('data/verreault_04012021.xlsx')
df_f = init('data/avram_june_03_01.xlsx')
#df_f = generate_file_for_cath_ef('data/zeeshan.xlsx')
#df_f = generate_file_for_cath_ef('data/goh.xlsx')
#df_f = generate_file_for_cath_ef('data/chung.xlsx')



display(df_f.LVEF.value_counts())

,L1 Intervention Performed?,L2 Intervention Performed?,L3 Intervention Performed?,L4 Intervention Performed?,L5 Intervention Performed?,L6 Intervention Performed?,L7 Intervention Performed?,L8 Intervention Performed?,L9 Intervention Performed?,L10 Intervention Performed?
0,No,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,No,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
734,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
735,No,No,No,No,No,No,No,NaN,NaN,NaN
736,Yes,Yes,Yes,No,No,NaN,NaN,NaN,NaN,NaN
737,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


KeyError: 'L1_Segment_Number'

In [70]:
df1 = pd.read_excel('data/data_generated_up_to_04012021/zeeshan_lvef.xlsx')
df1['Procedure_Date'] = df1['Procedure_Date'].astype(str)
display(df1['Procedure_Date'])
df2 = pd.read_csv('data/data_generated_up_to_04012021/avram_lvef.csv')

0      2020-12-10
1      2020-07-07
2      2020-09-03
3      2020-09-03
4      2020-07-13
          ...    
295    2020-08-27
296    2020-07-13
297    2020-07-05
298    2020-11-06
299    2020-12-24
Name: Procedure_Date, Length: 300, dtype: object

In [71]:
df1 = df1[['MRN','Procedure_Date']]
df2['MRN'] = df2['mrn']
df2 = df2[['MRN','Procedure_Date']]

In [72]:
df_concat = pd.concat([df1, df2])
display(df_concat.head(n=5))

,MRN,Procedure_Date
0,50017546,2020-12-10
1,10382224,2020-07-07
2,6370688,2020-09-03
3,6594683,2020-09-03
4,2080950,2020-07-13


In [73]:
df_concat.to_csv('data_pull_2_zeeshan_avram.csv')